# Test herdnet CBAM

## Installations

In [ ]:
# Check GPU
# !nvidia-smi

In [ ]:
# Install the dependencies
!pip install albumentations==1.0.3
!pip install fiftyone==0.14.3
!pip install hydra-core==1.1.0
!pip install opencv-python==4.5.1.48
!pip install pandas==1.2.3
!pip install pillow==8.2.0
!pip install scikit-image==0.18.1
!pip install scikit-learn==1.0.2
!pip install scipy==1.6.2
!pip install wandb==0.10.33

In [ ]:
# Download and install the code
import sys

!git clone https://github.com/victordpd19/HerdNet
!cd '/content/HerdNet' && python setup.py install

sys.path.append('/content/HerdNet')

## Create datasets

In [ ]:
#  some of the data of Delplanque et al. (2021) as an example
!gdown 1CcTAZZJdwrBfCPJtVH6VBU3luGKIN9st -O /content/data.zip
!unzip -oq /content/data.zip -d /content

In [ ]:
# Set the seed
from animaloc.utils.seed import set_seed

set_seed(9292)

In [ ]:
# Download cbam model
!gdown 1ri_1Nwvf72Bm1sh4FI3DMox2HoaCtR7H -O /content/cbam_model.pth


In [ ]:
device = torch.device('cpu')
checkpoint = torch.load("cbam_model.pth", map_location=device)
classes = checkpoint['classes']
num_classes = len(classes) + 1
img_mean = checkpoint['mean']
img_std = checkpoint['std']

In [ ]:
model= checkpoint['model']

In [ ]:
# Training, validation and test datasets
import albumentations as A
from animaloc.datasets import CSVDataset
from animaloc.data.transforms import DownSample

patch_size = 512
num_classes = 7
down_ratio = 2

albu_transforms = [A.Normalize(mean=img_mean, std=img_std)]

test_dataset = CSVDataset(
    csv_file = '/content/data/test.csv',
    root_dir = '/content/data/test',
    albu_transforms = albu_transforms,
    end_transforms = [DownSample(down_ratio=down_ratio, anno_type='point')]
    )



In [ ]:
# Dataloaders
from torch.utils.data import DataLoader

test_dataloader = DataLoader(dataset = test_dataset, batch_size = 1, shuffle = False)

## Create the Evaluator

In [ ]:
from torch.optim import Adam
from animaloc.train import Trainer
from animaloc.eval import PointsMetrics, HerdNetStitcher, HerdNetEvaluator
from animaloc.utils.useful_funcs import mkdir

work_dir = '/content/output'
mkdir(work_dir)

test_dir = '/content/test_output'
mkdir(test_dir)

metrics = PointsMetrics(radius=5, num_classes=num_classes) # radious 20

stitcher = HerdNetStitcher(
    model=model,
    size=(patch_size,patch_size),
    overlap=160,
    down_ratio=down_ratio,
    reduction='mean'
    device_name = device
    )

# Create an Evaluator
test_evaluator = HerdNetEvaluator(
    model=model,
    dataloader=test_dataloader,
    metrics=metrics,
    lmds_kwargs = dict(kernel_size=(3,3), adapt_ts=0.2, neg_ts=0.1), #lmds: local maximum detection strategy
    device_name = device,
    stitcher=stitcher,
    work_dir=test_dir,
    header='test'
    )

In [ ]:
test_f1_score = test_evaluator.evaluate(returns='f1_score')

In [ ]:
detections = test_evaluator.results
detections